In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub
import threading
import cv2
from ultralytics import YOLO
from datetime import datetime

# === Model Loading ===
yolo_model_path = 'yolo_v1 (2).pt'  # YOLO model path

# Load models
audio_model = tf.keras.models.load_model("best_model.keras")
yamnet_model=hub.load('https://www.kaggle.com/models/google/yamnet/TensorFlow2/yamnet/1')
yolo_model = YOLO(yolo_model_path)

# Ensure required directories exist
os.makedirs("anomalous_videos", exist_ok=True)
os.makedirs("anomalous_audio", exist_ok=True)

# Index-to-label mapping for YAMNet
index_to_label = {
    0: "Emergency_alert_sound",
    1: "Explosions",
    2:"Gunshots",
    3:"Human screams",
    4:"bottles breaking",
    5:"dog bark"
    # Add more labels as per your trained model
}

# Load and preprocess audio
def load_audio(file_path, target_sr=16000):
    """Load and preprocess audio to 16 kHz mono."""
    audio, sr = librosa.load(file_path, sr=target_sr, mono=True)
    return audio

# Preprocess and generate YAMNet embeddings
def generate_embeddings(yamnet_model, audio_waveforms):
    embeddings = []
    for waveform in audio_waveforms:
        processed_audio = preprocess_audio(waveform)  # Normalize and pad audio
        _, yamnet_embeddings, _ = yamnet_model(processed_audio)
        avg_embedding = tf.reduce_mean(yamnet_embeddings, axis=0)
        embeddings.append(avg_embedding.numpy())
    return np.array(embeddings)

def preprocess_audio(audio_waveform, target_length=16000 * 5):
    # Pad or truncate the waveform to the target length
    if len(audio_waveform) < target_length:
        audio_waveform = np.pad(audio_waveform, (0, target_length - len(audio_waveform)))
    else:
        audio_waveform = audio_waveform[:target_length]
    
    # Normalize between -1 and 1
    audio_waveform = audio_waveform.astype(np.float32)
    audio_waveform /= np.max(np.abs(audio_waveform))
    return audio_waveform


# Predict on a new audio file
def predict_audio(audio_file, confidence_threshold=0.5):
    # Load and preprocess the audio
    audio = load_audio(audio_file)
    processed_audio = preprocess_audio(audio)
    
    # Generate embeddings using YAMNet
    _, yamnet_embeddings, _ = yamnet_model(processed_audio)
    avg_embedding = tf.reduce_mean(yamnet_embeddings, axis=0).numpy().reshape(1, -1)
    
    # Make predictions
    prediction = audio_model.predict(avg_embedding)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = prediction[0][predicted_class_index]
    
    # Check confidence threshold
    if confidence < confidence_threshold:
        return "Unknown", confidence
    
    # Get class name
    predicted_class_name = index_to_label[predicted_class_index]
    return predicted_class_name, confidence


# === Video Threat Detection ===
def process_video_stream(video_source=0):
    cap = cv2.VideoCapture(video_source)  # Use 0 for webcam
    if not cap.isOpened():
        print("Error: Could not open video stream")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    out = None
    recording = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Warning: Failed to capture frame, skipping...")
            continue

        results = yolo_model.predict(frame, conf=0.5, verbose=False)
        annotated_frame = results[0].plot()

        anomalies_detected = False
        if results[0].boxes:
            for box in results[0].boxes:
                cls_id = int(box.cls)
                confidence = box.conf.item()
                label = yolo_model.names[cls_id]

                print(f"[{datetime.now()}] Detected: {label} (Confidence: {confidence:.2f})")
                if label in ["violence", "weaponized"]:
                    anomalies_detected = True

                    if not recording:
                        video_filename = f"anomalous_videos/{label}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
                        out = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))
                        recording = True
                        print(f"[{datetime.now()}] Recording started: {video_filename}")

                    if out:
                        out.write(frame)
                    break

        if not anomalies_detected and recording:
            print(f"[{datetime.now()}] Anomaly ended. Stopping recording.")
            recording = False
            if out:
                out.release()
                out = None

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit requested. Stopping program.")
            break

    if recording and out:
        out.release()
    cap.release()
    cv2.destroyAllWindows()

# === Unified System ===
def unified_system(audio_file=None, video_source=0):
    """Unified system for audio and video threat detection."""
    video_thread = threading.Thread(target=process_video_stream, args=(video_source,))
    audio_thread = None

    if audio_file:
        def audio_task():
            predicted_class, confidence = predict_audio(audio_file, confidence_threshold=0.7)
            # print(predicted_class, confidence)
            if predicted_class == "Unknown":
                print("The audio file does not belong to any trained class.")
            else:
                print(f"Predicted class: {predicted_class}")
                print(f"Confidence: {confidence:.2f}")

        audio_thread = threading.Thread(target=audio_task)
    video_thread.start()
    if audio_thread:
        audio_thread.start()
    video_thread.join()
    if audio_thread:
        audio_thread.join()

# Entry point
if __name__ == "__main__":
    audio_file_path = "R01-53-Medium-Sized Dog Barking.wav.mp3"  # Replace with your audio file path
    unified_system(audio_file=audio_file_path, video_source=0)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted class: dog bark
Confidence: 1.00
[2025-01-11 15:21:37.773687] Detected: non-violence (Confidence: 0.94)
[2025-01-11 15:21:37.874545] Detected: non-violence (Confidence: 0.92)
[2025-01-11 15:21:37.981393] Detected: non-violence (Confidence: 0.93)
[2025-01-11 15:21:38.083633] Detected: non-violence (Confidence: 0.89)
[2025-01-11 15:21:38.173591] Detected: non-violence (Confidence: 0.89)
[2025-01-11 15:21:38.283893] Detected: non-violence (Confidence: 0.87)
[2025-01-11 15:21:38.394760] Detected: non-violence (Confidence: 0.88)
[2025-01-11 15:21:38.489316] Detected: non-violence (Confidence: 0.90)
[2025-01-11 15:21:38.611215] Detected: non-violence (Confidence: 0.93)
[2025-01-11 15:21:38.689141] Detected: non-violence (Confidence: 0.95)
[2025-01-11 15:21:38.783421] Detected: non-violence (Confidence: 0.94)
[2025-01-11 15:21:38.889245] Detected: non-violence (Confidence: 0.94)
[2025-01-11 15:21:38.978494] Detected: non-violence (Confidence: 0.